## NLM Typo Correction Dataset

This script selects typo correction examples of the dataset (from the paper below) and puts them under `data/cspell_train`:

- An Ensemble Method for Spelling Correction in Consumer Health Questions

Please download `TrainingSet.tgz` from the webpage below and extract the file under `scripts/TrainingSet`.
https://lsg3.nlm.nih.gov/LexSysGroup/Projects/cSpell/current/docs/designDoc/UDF/TrainingSet/index.html

Spelling error type : Frequency
- NON-WORD: 436
- REAL-WORD: 154
- PUNCTUATION: 58
- TO-MERGE: 45
- TO-SPLIT: 315

In [1]:
import os
import sys
import re
import csv
import shutil
import random
import multiprocessing
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from tokenise import tokenize
# from collections import defaultdict, Counter
# from mimic_conv_notes import clean_text, sanitize_text

In [2]:
nlm_root = 'TrainingSet'
org_dir = os.path.join(nlm_root, 'OrgData.471')
real_std_dir = os.path.join(nlm_root, 'GoldStd-RealWord')
non_std_dir = os.path.join(nlm_root, 'GoldStd-NonWord')

output_dir = '../data/cspell_train'
output_fname = 'test.tsv'
output_char_only_fname = 'test_char.tsv'

In [4]:
note_fnames = os.listdir(org_dir)

In [5]:
test_idx = 200
fname = note_fnames[test_idx]
with open(os.path.join(org_dir, fname)) as fd:
    org_note = fd.read().strip()
with open(os.path.join(real_std_dir, fname)) as fd:
    real_note = fd.read().strip()
with open(os.path.join(non_std_dir, fname)) as fd:
    non_note = fd.read().strip()

In [6]:
print(org_note)
print('=' * 70)
print(real_note)
print('=' * 70)
print(non_note)

diagnosis of OI/ resources. I was writing to inquire about more information regarding the diagnosis of OI.  We have family members who are in the process of waiting for genetic testing to come back but are under allegations of child abuse.  Is there any information and / or resources that may be helpful to us? Any help is appreciated. Thank you,[NAME]
diagnosis of OI/ resources. I was writing to inquire about more information regarding the diagnosis of OI.  We have family members who are in the process of waiting for genetic testing to come back but are under allegations of child abuse.  Is there any information and / or resources that may be helpful to us? Any help is appreciated. Thank you, [NAME]
diagnosis of OI/ resources. I was writing to inquire about more information regarding the diagnosis of OI.  We have family members who are in the process of waiting for genetic testing to come back but are under allegations of child abuse.  Is there any information and / or resources that m

In [7]:
def get_unmatches(tokens1, tokens2):
    cost_arr = np.zeros((len(tokens1)+1, len(tokens2)+1), np.int32)
    oper_arr = np.zeros((len(tokens1), len(tokens2)), np.int32)
    matches = []
    unmatches = []
    for i1 in range(len(tokens1)):
        cost_arr[i1+1, 0] = i1+1
    for i2 in range(len(tokens2)):
        cost_arr[0, i2+1] = i2+1
    for i1 in range(len(tokens1)):
        t1 = tokens1[i1]
        for i2 in range(len(tokens2)):
            t2 = tokens2[i2]
            if t1 == t2:
                cost_arr[i1+1, i2+1] = cost_arr[i1, i2]
                oper_arr[i1, i2] = 0
            else:
                left, up, diag = cost_arr[i1+1, i2], cost_arr[i1, i2+1], cost_arr[i1, i2]
                if diag <= left and diag <= up:
                    cost_arr[i1+1, i2+1] = cost_arr[i1, i2] + 1
                    oper_arr[i1, i2] = 3  # sub
                elif left <= up:
                    cost_arr[i1+1, i2+1] = cost_arr[i1+1, i2] + 1
                    oper_arr[i1, i2] = 2  # add
                else:
                    cost_arr[i1+1, i2+1] = cost_arr[i1, i2+1] + 1
                    oper_arr[i1, i2] = 1  # del
    
    unmatch = [[], []]
    i1, i2 = len(tokens1)-1, len(tokens2)-1
    while True:
        if oper_arr[i1, i2] == 0:  # token match
            if len(unmatch[0]) + len(unmatch[1]) > 0:
                unmatches.insert(0, unmatch)
                unmatch = [[], []]
            i1, i2 = i1 - 1, i2 - 1
        elif oper_arr[i1, i2] == 1:  # del
            unmatch[0].insert(0, i1)
            i1, i2 = i1 - 1, i2
        elif oper_arr[i1, i2] == 2:  # add
            unmatch[1].insert(0, i2)
            i1, i2 = i1, i2 - 1
        else: # oper_arr[i1, i2] == 3  # sub
            unmatch[0].insert(0, i1)
            unmatch[1].insert(0, i2)
            i1, i2 = i1 - 1, i2 - 1
        if i1 < 0 or i2 < 0:
            if i1 >= 0:
                unmatch[0] = list(range(i1+1)) + unmatch[0]
            if i2 >= 0:
                unmatch[1] = list(range(i2+1)) + unmatch[1]
            if len(unmatch[0]) + len(unmatch[1]) > 0:
                unmatches.insert(0, unmatch)
                unmatch = [[], []]
            break
    return unmatches

In [8]:
def split(line):
    return [t for t in re.split(r'(;|,|\.|\?|-|\/|\(|\)|\[|\]|\s)\s*', line) if t.strip()]

In [9]:
examples = []
ex_id = 0
for fname in tqdm(note_fnames):
    if fname.endswith('.rm'):
        continue
    note_id = int(fname[:fname.index('.')])
    with open(os.path.join(org_dir, fname)) as fd:
        note_org = fd.read().strip()
    with open(os.path.join(real_std_dir, fname)) as fd:
        note_real = fd.read().strip()
    with open(os.path.join(non_std_dir, fname)) as fd:
        note_non = fd.read().strip()
        
    tokens_org = split(note_org)
    tokens_real = split(note_real)
    tokens_non = split(note_non)
    
    unmatches_real = get_unmatches(tokens_org, tokens_real)
#     unmatches_non = get_unmatches(tokens_org, tokens_non)
    
#     real_set = set([tuple(unmatch[0]) for unmatch in unmatches_real])
#     non_set = set([tuple(unmatch[0]) for unmatch in unmatches_non])
    
#     if non_set - real_set:
#         print(fname)
#         print(unmatches_real)
#         print(unmatches_non)
    for idxs_org, idxs_real in unmatches_real:
        if len(idxs_org) == 1 and len(idxs_real) == 1:
            typo_idx, correct_idx = idxs_org[0], idxs_real[0]
            typo = tokens_org[typo_idx].lower()
            correct = tokens_real[correct_idx].lower()
            left = ' '.join(tokens_real[:correct_idx])
            right = ' '.join(tokens_real[correct_idx+1:])
            examples.append((ex_id, note_id, typo, left, right, correct))
            ex_id += 1

100%|██████████| 472/472 [00:13<00:00, 35.26it/s]


In [10]:
len(examples)

511

In [11]:
examples[0]

(0,
 16829,
 'quary',
 '',
 'about PSA tests My grandfather have examined PSA test value is 0 . 430 ng / ml . His age is 86 years . My question is this value is normal for his health or harmful ?',
 'query')

In [17]:
for ex_id, note_id, typo, left, right, correct in examples:
    print(f'{typo} - {correct}')

quary - query
is - if
anklising - ankylosing
withdrawl - withdrawal
treatmet - treatment
condropathia - chondropathia
counterindicative - contraindicated
especialy - especially
eturn - return
hank - thank
plz - please
plz - please
que3tion: - question:
hitiala - hiatal
sindrome - syndrome
sindrome - syndrome
thing - think
stereotypy - stereotypic
prego - pregnancy
ostonecrois - osteonecrosis
crainosynstosis - craniosynostosis
metoptic - metopic
metoptic - metopic
sucken - sunken
medlin - medline
anmnio - amnio
im - i'm
syundrome - syndrome
guys - guy's
protombin - prothrombin
off - of
ehich - which
realgarding - regarding
regarfs - regards
urinaire - urinary
baging - begging
srep - strep
strp - strep
spot - stop
4 - of
clancy - clumsy
traumatologo - traumatologist
fasiciitis - fasciitis
fasiciitis - fasciitis
consistancy - consistency
alzheimers - alzheimer's
microsomnia - microsomia
miscrosomnia - microsomia
knoledge - knowledge
conversarion - conversion
dysmorphysm - dysmorphism
phli

In [18]:
correct

"can't"

In [19]:
ex_id, note_id, typo, left, right, correct = examples[100]
exclusive_chars = set(correct) ^ set(typo)
for c in exclusive_chars:
    print(f'{c} - {c.isalpha()}')

k - True
x - True
s - True


In [20]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [21]:
output_fpath = os.path.join(output_dir, output_fname)
with open(output_fpath, 'w') as fd:
    writer = csv.writer(fd, delimiter='\t')
    writer.writerow(['index', 'note_id', 'word', 'left', 'right', 'correct'])  
    for ex in examples:
        writer.writerow(ex)

In [22]:
output_char_only_fpath = os.path.join(output_dir, output_char_only_fname)
with open(output_char_only_fpath, 'w') as fd:
    writer = csv.writer(fd, delimiter='\t')
    writer.writerow(['index', 'note_id', 'word', 'left', 'right', 'correct'])  
    for ex in examples:
        ex_id, note_id, typo, left, right, correct = ex
        exclusive_chars = set(correct) ^ set(typo)
        temp = False
        for c in exclusive_chars:
            temp = temp | c.isalpha()
        if temp:
            continue
        writer.writerow(ex)

In [23]:
fname = '1.txt'
note_id = int(fname[:fname.index('.')])
with open(os.path.join(org_dir, fname)) as fd:
    note_org = fd.read().strip()
with open(os.path.join(real_std_dir, fname)) as fd:
    note_real = fd.read().strip()
with open(os.path.join(non_std_dir, fname)) as fd:
    note_non = fd.read().strip()

tokens_org = split(note_org)
tokens_real = split(note_real)
tokens_non = split(note_non)

unmatches_real = get_unmatches(tokens_org, tokens_real)
unmatches_non = get_unmatches(tokens_org, tokens_non)

In [24]:
print(unmatches_real)

for i1s, i2s in unmatches_real:
    tt1 = [tokens_org[i1] for i1 in i1s]
    tt2 = [tokens_real[i2] for i2 in i2s]
    print(f"{tt1} - {tt2}: {''.join(tt1) == ''.join(tt2)}")

[[[8], [8]], [[17], [17]], [[19, 20, 21], [19, 20]], [[27], [26]], [[42], [41]], [[45], [44]]]
['anixity'] - ['anxiety']: False
['dianosed'] - ['diagnosed']: False
['on', 'set', 'deminita'] - ['onset', 'dementia']: False
['yall'] - ["y'all"]: False
['seretona'] - ['serotonin']: False
['nonething'] - ['nothing']: False


In [25]:
print(unmatches_non)

for i1s, i2s in unmatches_non:
    tt1 = [tokens_org[i1] for i1 in i1s]
    tt2 = [tokens_non[i2] for i2 in i2s]
    print(f"{tt1} - {tt2}: {''.join(tt1) == ''.join(tt2)}")

[[[8], [8]], [[17], [17]], [[21], [21]], [[27], [27]], [[42], [42]], [[45], [45]]]
['anixity'] - ['anxiety']: False
['dianosed'] - ['diagnosed']: False
['deminita'] - ['dementia']: False
['yall'] - ["y'all"]: False
['seretona'] - ['serotonin']: False
['nonething'] - ['nothing']: False


In [26]:
note_org

'My mom is 82 years old suffering from anixity and depression for the last 10 years was dianosed early on set deminita 3 years ago. Do yall have a office in Greensboro NC? Can you recommend someone. she has seretona syndrome and nonething helps her.'

In [27]:
note_real

"My mom is 82 years old suffering from anxiety and depression for the last 10 years was diagnosed early onset dementia 3 years ago. Do y'all have a office in Greensboro NC? Can you recommend someone. she has serotonin syndrome and nothing helps her."